This is my capstone project. During the Covid 19 pandemic shutdown, I want to find out where to go for grocery shopping and dinner take-out. The project will  help me find neighborhoods in Bergen County with low confirmed caronovirus cases and more open restaurants/grocery stores.


In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


In [2]:
# Getting coordinates for the towns in Bergen county, NJ
!pip install geopandas
!pip install geopy

     |████████████████████████████████| 931kB 7.3MB/s eta 0:00:01
     |████████████████████████████████| 1.8MB 40.5MB/s eta 0:00:01
     |████████████████████████████████| 14.7MB 13.2MB/s eta 0:00:01
     |████████████████████████████████| 10.9MB 27.4MB/s eta 0:00:01


In [3]:
# Test geocode for one borough
from geopy.geocoders import Nominatim
locator = Nominatim(user_agent="myGeocoder")
location = locator.geocode("11 Rising Ridge Rd, Upper Saddle River, NJ, USA")
home_latitude = location.latitude
home_longitude = location.longitude
print("Home Latitude = {}, Longitude = {}".format(home_latitude, home_longitude))

Home Latitude = 41.08037993450633, Longitude = -74.10046599629999


In [4]:
import pyproj

geod = pyproj.Geod(ellps='WGS84')
location = locator.geocode("Hackensack, NJ, USA")
azimuth1, azimuth2, distance = geod.inv(home_latitude, location.latitude, home_longitude, location.longitude)
distance = int(distance // 1000)
print(distance)

15088


In [5]:
# import file 'Bergen_County_Covid19_Cases.csv' from local dataset
import types
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_48cbe6a955574f42bdf293db9085cbef = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='JwlV4cFFm0S7Cp3G3xvu-qo_QG12PjS4pKLjPZWZhfLr',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_48cbe6a955574f42bdf293db9085cbef.get_object(Bucket='courseracapstone-donotdelete-pr-bqfxymcsrfz1o5',Key='Bergen_County_Covis19_Cases.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
df_borough = pd.read_csv(body)
df_borough.head()


,Borough,Coronavirus_Positive_Cases
0,Allendale,54
1,Alpine,21
2,Bergenfield,726
3,Bogota,162
4,Carlstadt,89


In [6]:
latitude = [] 
longitude = []

for value in df_borough["Borough"]: 
    locator = Nominatim(user_agent="myGeocoder")
    location = locator.geocode(value + ", NJ, USA")
    latitude.append(location.latitude)
    longitude.append(location.longitude)
    print("{}: {},{}".format(value, location.latitude, location.longitude))
       
df_borough["Latitude"] = latitude   
df_borough["Longitude"] = longitude   
df_borough.head()


Allendale: 41.0414858,-74.129032
Alpine: 40.94855,-73.9281139
Bergenfield: 40.9275987,-73.9973608
Bogota: 40.8762106,-74.0298621
Carlstadt: 40.8403778,-74.0906974
Cliffside Park: 40.8214894,-73.9876388
Closter: 40.9731536,-73.9615262
Cresskill: 40.9414874,-73.9593041
Demarest: 40.9573205,-73.9634708
Dumont: 40.9406541,-73.9968051
East Rutherford: 40.833989,-74.0970865
Edgewater: 40.8270448,-73.975694
Elmwood Park: 40.9039878,-74.118476
Emerson: 40.976209,-74.0262505
Englewood: 40.8928771,-73.9726381
Englewood Cliffs: 40.8853773,-73.9523597
Fair Lawn: 40.9403762,-74.1318096
Fairview: 40.8126006,-73.999028
Fort Lee: 40.8509333,-73.9701382
Franklin Lakes: 41.0167639,-74.2057012
Garfield: 40.8814882,-74.113198
Glen Rock: 40.9628758,-74.1329208
Hackensack: 40.8871438,-74.0410865
Harrington Park: 40.9837089,-73.9798601
Hasbrouck Heights: 40.8579293,-74.0816693
Haworth: 40.9609315,-73.9901383
Hillsdale: 41.0025975,-74.0404175
Ho-Ho-Kus: 40.9976524,-74.110284
Leonia: 40.8617353,-73.9880495
Lit

,Borough,Coronavirus_Positive_Cases,Latitude,Longitude
0,Allendale,54,41.041486,-74.129032
1,Alpine,21,40.948550,-73.928114
2,Bergenfield,726,40.927599,-73.997361
3,Bogota,162,40.876211,-74.029862
4,Carlstadt,89,40.840378,-74.090697


In [7]:
print('Total {} boroughs'.format(df_borough.shape[0]))

Total 70 boroughs


In [8]:
# Hackensack is the center of Bergen County, NJ
locator = Nominatim(user_agent="myGeocoder")
location = locator.geocode("Hackensack, NJ, USA")
county_latitude = location.latitude
county_longitude = location.longitude

In [9]:
!pip install folium==0.5.0
import folium 

     |████████████████████████████████| 81kB 7.2MB/s eta 0:00:011
  Stored in directory: /home/dsxuser/.cache/pip/wheels/f8/98/ff/954791afc47740d554f0d9e5885fa09dd60c2265d42578e665
Successfully built folium


In [47]:
# create map of Toronto using latitude and longitude values
map_bergen_county = folium.Map(location=[county_latitude, county_longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, case in zip(df_borough['Latitude'], df_borough['Longitude'], df_borough['Borough'], df_borough['Coronavirus_Positive_Cases']):
    label = '{}: [{} cases]'.format(borough, case)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bergen_county)  
    
map_bergen_county

In [11]:
CLIENT_ID = 'DTI3F3LYBQK3MGGWQFEWIZGSQWKUC0ON4ZOOZXQPFWPOJPXC' # your Foursquare ID
CLIENT_SECRET = 'CDLNKMXNCTLPUKSXHCBWUE3CRLQ1RAYQFPCY0W53YPJI0DUI' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
ACCESS_TOKEN = 'X10MOF5IU5WTDA3QC0ORNA4GULI2N3EF4E1UYRU0DS0SNOKZ'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
print('ACCESS_TOKEN:' + ACCESS_TOKEN)

Your credentails:
CLIENT_ID: DTI3F3LYBQK3MGGWQFEWIZGSQWKUC0ON4ZOOZXQPFWPOJPXC
CLIENT_SECRET:CDLNKMXNCTLPUKSXHCBWUE3CRLQ1RAYQFPCY0W53YPJI0DUI
ACCESS_TOKEN:X10MOF5IU5WTDA3QC0ORNA4GULI2N3EF4E1UYRU0DS0SNOKZ


In [12]:
# Test FourSquare API with neighborhood data
neighborhood_latitude = 40.888356
neighborhood_longitude = -74.040095
neighborhood_name = 'Hackensack'


In [13]:
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=DTI3F3LYBQK3MGGWQFEWIZGSQWKUC0ON4ZOOZXQPFWPOJPXC&client_secret=CDLNKMXNCTLPUKSXHCBWUE3CRLQ1RAYQFPCY0W53YPJI0DUI&ll=40.888356,-74.040095&v=20180605&radius=500&limit=100'

In [14]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eb209b347e0d6001b5f455e'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Hackensack',
  'headerFullLocation': 'Hackensack',
  'headerLocationGranularity': 'city',
  'totalResults': 30,
  'suggestedBounds': {'ne': {'lat': 40.892856004500004,
    'lng': -74.0341536208967},
   'sw': {'lat': 40.8838559955, 'lng': -74.04603637910328}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b968656f964a52022d234e3',
       'name': 'Bangkok Garden',
       'location': {'address': '261 Main St',
        'lat': 40.88649410842714,
        'lng': -74.04159271838799,
        'labeledLatLngs': [{'label': 'display',
      

In [15]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [16]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.id', 'venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,id,name,categories,lat,lng
0,4b968656f964a52022d234e3,Bangkok Garden,Thai Restaurant,40.886494,-74.041593
1,4a9616eaf964a520fc2520e3,White Manna Hamburgers,Burger Joint,40.889151,-74.038539
2,4e21a424e4cdf68591870f3e,Colombia Bakery,Bakery,40.889106,-74.040124
3,578bc1a6498e4c52e110d85e,Noches De Colombia,Colombian Restaurant,40.890476,-74.039506
4,4c53850bb3b09c74a1b613b4,Bohemia Restaurant,Latin American Restaurant,40.887356,-74.041186
5,4c040a039a7920a138b6d079,Citrus Cafe,South American Restaurant,40.887911,-74.040846
6,4bb646b71344b71305419d04,Giant Farmers Market,Farmers Market,40.888356,-74.040095
7,4bc0a0ca4cdfc9b6a61e9321,Birkenmeier Sport Shop,Sporting Goods Shop,40.885433,-74.042643
8,4cb746bd3ac937040273da0a,Java's Brewin',Coffee Shop,40.886801,-74.041382
9,4bc37d1174a9a5935e9fd4f6,Ruben's Smoke Stack,Smoke Shop,40.888668,-74.040403


In [17]:
def getNearbyVenues(names, cases, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, case, lat, lng in zip(names, cases, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            case,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['id'],
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Coronavirus Positive Cases',
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue id',
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
bergen_venues = getNearbyVenues(names=df_borough['Borough'],
                                cases=df_borough['Coronavirus_Positive_Cases'],
                                latitudes=df_borough['Latitude'],
                                longitudes=df_borough['Longitude']
                                )


Allendale
Alpine
Bergenfield
Bogota
Carlstadt
Cliffside Park
Closter
Cresskill
Demarest
Dumont
East Rutherford
Edgewater
Elmwood Park
Emerson
Englewood
Englewood Cliffs
Fair Lawn
Fairview
Fort Lee
Franklin Lakes
Garfield
Glen Rock
Hackensack
Harrington Park
Hasbrouck Heights
Haworth
Hillsdale
Ho-Ho-Kus
Leonia
Little Ferry
Lodi
Lyndhurst
Mahwah
Maywood
Midland Park
Montvale
Moonachie
New Milford
North Arlington
Northvale
Norwood
Oakland
Old Tappan
Oradell
Palisades Park
Paramus
Park Ridge
Ramsey
Ridgefield
Ridgefield Park
Ridgewood
River Edge
River Vale
Rochelle Park
Rockleigh
Rutherford
Saddle Brook
Saddle River
South Hackensack
Teaneck
Tenafly
Teterboro
Upper Saddle River
Waldwick
Wallington
Washington Twp.
Westwood
Wood-Ridge
Woodcliff Lake
Wyckoff


In [19]:
print(bergen_venues.shape)
bergen_venues.head()

(1396, 9)


,Neighborhood,Coronavirus Positive Cases,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue id,Venue Latitude,Venue Longitude,Venue Category
0,Bergenfield,726,40.927599,-73.997361,Chapala Grill,4baff2e6f964a520b02d3ce3,40.926046,-73.996719,Mexican Restaurant
1,Bergenfield,726,40.927599,-73.997361,Christine's Bake Shop,4b49f4d3f964a520b07626e3,40.928778,-73.997970,Bakery
2,Bergenfield,726,40.927599,-73.997361,Tommy Fox's,4b3617f3f964a520163025e3,40.926707,-73.996453,Pub
3,Bergenfield,726,40.927599,-73.997361,U.M.A. United Martial Arts Taekwondo,535acd65498e70b6d315b6fc,40.926948,-73.996311,Martial Arts Dojo
4,Bergenfield,726,40.927599,-73.997361,Empanada Mania,4ed92ffb6c2547bc612ccc96,40.925710,-73.996562,Empanada Restaurant


In [20]:
print('There are {} uniques categories.'.format(len(bergen_venues['Venue Category'].unique())))

There are 218 uniques categories.


In [34]:
bergen_venues['Venue Category'].unique()

array(['Mexican Restaurant', 'Bakery', 'Pub', 'Martial Arts Dojo',
       'Empanada Restaurant', 'Pizza Place', 'Burger Joint',
       'Discount Store', 'Bagel Shop', 'Asian Restaurant',
       'Movie Theater', 'Bar', 'Pharmacy', 'Convenience Store',
       'Optical Shop', 'Sandwich Place', 'Japanese Restaurant', 'Gym',
       'BBQ Joint', 'Bus Line', 'Italian Restaurant',
       'Gym / Fitness Center', 'Bank', 'Latin American Restaurant',
       'Chinese Restaurant', 'Spa', 'Liquor Store', 'Cosmetics Shop',
       'Deli / Bodega', 'Breakfast Spot', 'Restaurant', 'Café',
       'Middle Eastern Restaurant', 'Steakhouse', 'Bus Stop',
       'Massage Studio', 'Doner Restaurant', 'Turkish Restaurant',
       'Supermarket', 'Indian Restaurant', 'Ramen Restaurant',
       'Grocery Store', 'Malay Restaurant', 'Donut Shop',
       'Thai Restaurant', 'Korean Restaurant', 'Russian Restaurant',
       'Coffee Shop', 'Vietnamese Restaurant', 'Dessert Shop',
       'Supplement Shop', 'Big Box Store

In [75]:
# Filter by restaurant
# column_filters = {'Venue Category': ['Restaurant', 'Cafe', 'Bakery', 'Pizza', 'Diner', 'Sandwich', 'Deli', 'Steakhouse']}
column_filters = {'Venue Category': ['Restaurant', 'Pizza']}
for k in column_filters.keys():
    column_filters[k] = '|'.join(column_filters[k])
column_filters

bergen_restaurant = bergen_venues[bergen_venues['Venue Category'].str.contains(column_filters['Venue Category'], case=False)].reset_index(drop=True)

bergen_restaurant.shape

(475, 9)

In [76]:
bergen_restaurant.head()

,Neighborhood,Coronavirus Positive Cases,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue id,Venue Latitude,Venue Longitude,Venue Category
0,Bergenfield,726,40.927599,-73.997361,Chapala Grill,4baff2e6f964a520b02d3ce3,40.926046,-73.996719,Mexican Restaurant
1,Bergenfield,726,40.927599,-73.997361,Empanada Mania,4ed92ffb6c2547bc612ccc96,40.925710,-73.996562,Empanada Restaurant
2,Bergenfield,726,40.927599,-73.997361,Nick's Pizza,4b8feb35f964a520886933e3,40.927700,-73.998180,Pizza Place
3,Bergenfield,726,40.927599,-73.997361,Cusinera,4c26424a136d20a1a1fee461,40.925610,-73.996678,Asian Restaurant
4,Bergenfield,726,40.927599,-73.997361,Bamboo Grill,4b7c8698f964a520ce982fe3,40.925985,-73.996648,Asian Restaurant


In [23]:
# Filter by supermarket
column_filters = {'Venue Category': ['Supermarket', 'Grocery', 'Farmers Market']}
for k in column_filters.keys():
    column_filters[k] = '|'.join(column_filters[k])
column_filters

bergen_supermarket = bergen_venues[bergen_venues['Venue Category'].str.contains(column_filters['Venue Category'], case=False)].reset_index(drop=True)

bergen_supermarket

,Neighborhood,Coronavirus Positive Cases,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue id,Venue Latitude,Venue Longitude,Venue Category
0,Cliffside Park,415,40.821489,-73.987639,Food Bazaar Supermarket,5840de9caf5c144c4e2e3e38,40.821489,-73.991550,Supermarket
1,Cliffside Park,415,40.821489,-73.987639,Nizam Turkish Supermarket,4d23aa4175526dcb8bf102a9,40.825232,-73.987794,Grocery Store
2,Closter,72,40.973154,-73.961526,Stop & Shop,5634b031498e1a42e18cc7bf,40.969031,-73.962986,Supermarket
3,Cresskill,96,40.941487,-73.959304,Kings Food Markets,4bd38634a8b3a593003f6a5f,40.940876,-73.960311,Supermarket
4,Dumont,278,40.940654,-73.996805,Super Stop & Shop,4b9925b3f964a520b66435e3,40.937412,-73.996043,Supermarket
5,East Rutherford,123,40.833989,-74.097087,Aldi's,4e9d913a4690bdef1031f654,40.831393,-74.092718,Supermarket
6,Edgewater,119,40.827045,-73.975694,Whole Foods Market,4a6e34ccf964a52034d41fe3,40.824706,-73.973828,Grocery Store
7,Englewood,690,40.892877,-73.972638,ShopRite of Englewood,4b0fe28bf964a520d46523e3,40.896316,-73.975366,Grocery Store
8,Englewood,690,40.892877,-73.972638,Englewood Farmer's Market,4c5354bc30f92d7f348897b8,40.897163,-73.973404,Farmers Market
9,Garfield,669,40.881488,-74.113198,Price Rite Of Garfield,53851be1498eb1110eeb830e,40.881520,-74.116071,Grocery Store


In [24]:
# search detail for one venue based on venue id
# https://api.foursquare.com/v2/venues/venue_id?client_id=client_id&client_secret=client_secret&v=version
venue_id = '4b5762e8f964a520243528e3'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}&oauth_token={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, ACCESS_TOKEN)
url

'https://api.foursquare.com/v2/venues/4b5762e8f964a520243528e3?client_id=DTI3F3LYBQK3MGGWQFEWIZGSQWKUC0ON4ZOOZXQPFWPOJPXC&client_secret=CDLNKMXNCTLPUKSXHCBWUE3CRLQ1RAYQFPCY0W53YPJI0DUI&v=20180605&oauth_token=X10MOF5IU5WTDA3QC0ORNA4GULI2N3EF4E1UYRU0DS0SNOKZ'

In [25]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eb20955882fc7001b69d34d'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venue': {'id': '4b5762e8f964a520243528e3',
   'name': 'Whole Foods Market',
   'contact': {'phone': '2016700383',
    'formattedPhone': '(201) 670-0383',
    'twitter': 'wfmridgewood',
    'facebook': '24922591487',
    'facebookUsername': 'wholefoodsmarket',
    'facebookName': 'Whole Foods Market'},
   'location': {'address': '44 Godwin Ave',
    'crossStreet': 'Wilsey Sq',
    'lat': 40.978417587813986,
    'lng': -74.12259307781991,
    'labeledLatLngs': [{'label': 'display',
      'lat': 40.978417587813986,
      'lng': -74.12259307781991}],
    'postalCode': '07450',
    'cc': 'US',
    'city': 'Ridgewood',
    'state': 'NJ',
    'country': 'United States',
    'formattedAddress': ['44 Godwin Ave (Wilsey Sq)', 'Ridgewood, NJ 07450']},
   'canonicalUrl': 'https://foursquare.com/v/whole-foods-market/4b5762e8f964a520243528e3',


In [26]:
try:
    print(results['response']['venue']['rating'], results['response']['venue']['hours']['isOpen'])
except:
    print('This venue has not been rated yet.')

7.9 False


In [31]:
def getRatingDistanceOpenStatus(boroughs, cases, venues, venue_ids, categories, venue_latitudes, venue_longitudes):
    
    venues_list=[]
    for borough, case, venue, venue_id, category, venue_latitude, venue_longitude in zip(boroughs, cases, venues, venue_ids, categories, venue_latitudes, venue_longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}&oauth_token={}'.format(
            venue_id, 
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            ACCESS_TOKEN)
            
        # make the GET request
        results = requests.get(url).json()
        
        # get rating and open status
        try:
            rating = results['response']['venue']['rating']
        except:
            rating = 'N/A'
        
        try:
            openStatus = results['response']['venue']['hours']['status']
        except:
            openStatus = 'N/A'
        
        # Calculate distance to my home
        azimuth1, azimuth2, distance = geod.inv(home_latitude, venue_latitude, home_longitude, venue_longitude)
        distance = int(distance // 1000)
        print('{} {} {} {}'.format(venue, rating, distance, openStatus))
        
        # filter our venues without rating or venues not open
        if rating != 'N/A' and openStatus != 'N/A':
            venues_list.append([(
            venue, 
            category,
            borough, 
            case,
            rating,
            distance,
            venue_latitude,
            venue_longitude,
            openStatus)])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [
                    'Venue', 
                    'Venue Category',
                    'Borough', 
                    'Coronavirus Positive Cases',
                    'Rating',
                    'Distance',
                    'Venue Latitude',
                    'Venue Longitude',
                    'Open Status']
    
    return(nearby_venues)

In [39]:
# Work on supermarket clustering
bergen_open_supermarket = getRatingDistanceOpenStatus(boroughs=bergen_supermarket['Neighborhood'],
                                cases=bergen_supermarket['Coronavirus Positive Cases'],
                                venues=bergen_supermarket['Venue'],
                                venue_ids=bergen_supermarket['Venue id'],
                                venue_latitudes=bergen_supermarket['Venue Latitude'],
                                venue_longitudes=bergen_supermarket['Venue Longitude'],                 
                                categories=bergen_supermarket['Venue Category']
                                )
bergen_open_supermarket.head()

Food Bazaar Supermarket 8.0 15082 Open until 10:30 PM
Nizam Turkish Supermarket 7.9 15083 N/A
Stop & Shop 5.9 15098 Closed until 7:30 AM tomorrow
Kings Food Markets 7.5 15095 N/A
Super Stop & Shop 6.8 15094 Closed until 7:30 AM tomorrow
Aldi's 7.2 15081 Closed until 9:00 AM tomorrow
Whole Foods Market 8.5 15083 Closed until 9:00 AM tomorrow
ShopRite of Englewood 7.2 15090 Open
Englewood Farmer's Market N/A 15090 N/A
Price Rite Of Garfield 6.9 15086 Open until 10:00 PM
Kilroy's Wonder Market 7.6 15094 Closed until 8:00 AM tomorrow
Giant Farmers Market 7.3 15088 N/A
Sunrise Oriental Products N/A 15088 N/A
Kings Food Markets 7.6 15100 Open until 10:00 PM
H-Mart 6.5 15086 N/A
ShopRite of Lodi 7.3 15086 Open until 11:59 PM
ShopRite 6.9 15079 Open until Midnight
Quick Mart N/A 15083 N/A
Krauszers 7.7 15098 N/A
Trader Joe's 9.2 15094 Closed until 9:00 AM tomorrow
Ramsey Farmer's Market 8.8 15103 Closed until 9:00 AM Sunday
H&Y Marketplace 6.4 15083 N/A
Village IGA 5.5 15085 N/A
Whole Foods Ma

,Venue,Venue Category,Borough,Coronavirus Positive Cases,Rating,Distance,Venue Latitude,Venue Longitude,Open Status
0,Food Bazaar Supermarket,Supermarket,Cliffside Park,415,8.0,15082,40.821489,-73.991550,Open until 10:30 PM
1,Stop & Shop,Supermarket,Closter,72,5.9,15098,40.969031,-73.962986,Closed until 7:30 AM tomorrow
2,Super Stop & Shop,Supermarket,Dumont,278,6.8,15094,40.937412,-73.996043,Closed until 7:30 AM tomorrow
3,Aldi's,Supermarket,East Rutherford,123,7.2,15081,40.831393,-74.092718,Closed until 9:00 AM tomorrow
4,Whole Foods Market,Grocery Store,Edgewater,119,8.5,15083,40.824706,-73.973828,Closed until 9:00 AM tomorrow


In [44]:
print('Total supermarkets: {}'.format(bergen_supermarket.shape[0]))
print('Total open supermarkets: {}'.format(bergen_open_supermarket.shape[0]))

Total supermarkets: 32
Total open supermarkets: 19


In [40]:
bergen_open_supermarket.sort_values(['Coronavirus Positive Cases', 'Rating', 'Distance'], ascending=[True, False, True])

,Venue,Venue Category,Borough,Coronavirus Positive Cases,Rating,Distance,Venue Latitude,Venue Longitude,Open Status
1,Stop & Shop,Supermarket,Closter,72,5.9,15098,40.969031,-73.962986,Closed until 7:30 AM tomorrow
17,Super Foodtown,Supermarket,Washington Twp.,89,7.3,15098,40.986631,-74.064956,Open until 10:00 PM
7,Kilroy's Wonder Market,Grocery Store,Glen Rock,99,7.6,15094,40.962113,-74.127409,Closed until 8:00 AM tomorrow
8,Kings Food Markets,Grocery Store,Hillsdale,101,7.6,15100,41.001234,-74.041039,Open until 10:00 PM
12,Ramsey Farmer's Market,Farmers Market,Ramsey,109,8.8,15103,41.056289,-74.142212,Closed until 9:00 AM Sunday
4,Whole Foods Market,Grocery Store,Edgewater,119,8.5,15083,40.824706,-73.973828,Closed until 9:00 AM tomorrow
3,Aldi's,Supermarket,East Rutherford,123,7.2,15081,40.831393,-74.092718,Closed until 9:00 AM tomorrow
16,Stop & Shop,Supermarket,Tenafly,141,5.7,15094,40.926974,-73.965214,Closed until 7:30 AM tomorrow
18,Trader Joe's,Grocery Store,Westwood,177,8.3,15099,40.992950,-74.035835,Closed until 10:00 AM tomorrow
13,Whole Foods Market,Grocery Store,Ridgewood,262,7.9,15096,40.978418,-74.122593,Closed until 9:00 AM tomorrow


In [41]:
# clustering venues using K-Means

# set number of clusters
kclusters = 5

bergen_supermarket_clustering = bergen_open_supermarket[['Coronavirus Positive Cases', 'Rating', 'Distance']]

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bergen_supermarket_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([4, 0, 2, 0, 0, 1, 1, 0, 0, 1, 2, 3, 0, 2, 2, 2, 0, 0, 0],
      dtype=int32)

In [42]:
# add clustering labels
bergen_open_supermarket.insert(2, 'Cluster Labels', kmeans.labels_)

bergen_open_supermarket.head()

,Venue,Venue Category,Cluster Labels,Borough,Coronavirus Positive Cases,Rating,Distance,Venue Latitude,Venue Longitude,Open Status
0,Food Bazaar Supermarket,Supermarket,4,Cliffside Park,415,8.0,15082,40.821489,-73.991550,Open until 10:30 PM
1,Stop & Shop,Supermarket,0,Closter,72,5.9,15098,40.969031,-73.962986,Closed until 7:30 AM tomorrow
2,Super Stop & Shop,Supermarket,2,Dumont,278,6.8,15094,40.937412,-73.996043,Closed until 7:30 AM tomorrow
3,Aldi's,Supermarket,0,East Rutherford,123,7.2,15081,40.831393,-74.092718,Closed until 9:00 AM tomorrow
4,Whole Foods Market,Grocery Store,0,Edgewater,119,8.5,15083,40.824706,-73.973828,Closed until 9:00 AM tomorrow


In [56]:
# create map for supermarket clustering
map_supermarket_clusters = folium.Map(location=[county_latitude, county_longitude], zoom_start=11)

# add my home
folium.CircleMarker(
        [home_latitude, home_longitude],
        radius=9,
        popup='Home',
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_supermarket_clusters)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, venue, cluster, cases, rating in zip(bergen_open_supermarket['Venue Latitude'], bergen_open_supermarket['Venue Longitude'], bergen_open_supermarket['Venue'], bergen_open_supermarket['Cluster Labels'], bergen_open_supermarket['Coronavirus Positive Cases'], bergen_open_supermarket['Rating']):
    label = folium.Popup(str(venue) + ' [cluster: ' + str(cluster) + ', Coronavirus cases: ' + str(cases) + ', Rating: ' + str(rating) + ']', parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=6,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_supermarket_clusters)
       
map_supermarket_clusters

In [61]:
# supermarket - cluster 0 - low coronuavirus cases
bergen_open_supermarket.loc[bergen_open_supermarket['Cluster Labels'] == 0, bergen_open_supermarket.columns[[0]+[1]+[3]+[4]+[5]+[6]]]

,Venue,Venue Category,Borough,Coronavirus Positive Cases,Rating,Distance
1,Stop & Shop,Supermarket,Closter,72,5.9,15098
3,Aldi's,Supermarket,East Rutherford,123,7.2,15081
4,Whole Foods Market,Grocery Store,Edgewater,119,8.5,15083
7,Kilroy's Wonder Market,Grocery Store,Glen Rock,99,7.6,15094
8,Kings Food Markets,Grocery Store,Hillsdale,101,7.6,15100
12,Ramsey Farmer's Market,Farmers Market,Ramsey,109,8.8,15103
16,Stop & Shop,Supermarket,Tenafly,141,5.7,15094
17,Super Foodtown,Supermarket,Washington Twp.,89,7.3,15098
18,Trader Joe's,Grocery Store,Westwood,177,8.3,15099


In [62]:
# supermarket - cluster 1 - high coronavirus cases and low rating
bergen_open_supermarket.loc[bergen_open_supermarket['Cluster Labels'] == 1, bergen_open_supermarket.columns[[0]+[1]+[3]+[4]+[5]+[6]]]

,Venue,Venue Category,Borough,Coronavirus Positive Cases,Rating,Distance
5,ShopRite of Englewood,Grocery Store,Englewood,690,7.2,15090
6,Price Rite Of Garfield,Grocery Store,Garfield,669,6.9,15086
9,ShopRite of Lodi,Grocery Store,Lodi,574,7.3,15086


In [63]:
# supermarket - cluster 2 - medium coronavirus cases and low rating
bergen_open_supermarket.loc[bergen_open_supermarket['Cluster Labels'] == 2, bergen_open_supermarket.columns[[0]+[1]+[3]+[4]+[5]+[6]]]

,Venue,Venue Category,Borough,Coronavirus Positive Cases,Rating,Distance
2,Super Stop & Shop,Supermarket,Dumont,278,6.8,15094
10,ShopRite,Supermarket,Lyndhurst,331,6.9,15079
13,Whole Foods Market,Grocery Store,Ridgewood,262,7.9,15096
14,Super Stop & Shop,Supermarket,Ridgewood,262,5.8,15096
15,Acme,Supermarket,Saddle Brook,267,7.1,15088


In [64]:
# supermarket - cluster 3 - high coronavirus cases and high rating
bergen_open_supermarket.loc[bergen_open_supermarket['Cluster Labels'] == 3, bergen_open_supermarket.columns[[0]+[1]+[3]+[4]+[5]+[6]]]

,Venue,Venue Category,Borough,Coronavirus Positive Cases,Rating,Distance
11,Trader Joe's,Grocery Store,Paramus,831,9.2,15094


In [65]:
# supermarket - cluster 4 - medium coronavirus cases and high rating
bergen_open_supermarket.loc[bergen_open_supermarket['Cluster Labels'] == 4, bergen_open_supermarket.columns[[0]+[1]+[3]+[4]+[5]+[6]]]

,Venue,Venue Category,Borough,Coronavirus Positive Cases,Rating,Distance
0,Food Bazaar Supermarket,Supermarket,Cliffside Park,415,8.0,15082


In [92]:
# Filter by restaurant
# column_filters = {'Venue Category': ['Restaurant', 'Cafe', 'Bakery', 'Pizza', 'Diner', 'Sandwich', 'Deli', 'Steakhouse']}
column_filters = {'Venue Category': ['Restaurant', 'Pizza']}
for k in column_filters.keys():
    column_filters[k] = '|'.join(column_filters[k])
column_filters

bergen_restaurant = bergen_venues[bergen_venues['Venue Category'].str.contains(column_filters['Venue Category'], case=False)].reset_index(drop=True)

bergen_restaurant.head(40)

,Neighborhood,Coronavirus Positive Cases,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue id,Venue Latitude,Venue Longitude,Venue Category
0,Bergenfield,726,40.927599,-73.997361,Chapala Grill,4baff2e6f964a520b02d3ce3,40.926046,-73.996719,Mexican Restaurant
1,Bergenfield,726,40.927599,-73.997361,Empanada Mania,4ed92ffb6c2547bc612ccc96,40.925710,-73.996562,Empanada Restaurant
2,Bergenfield,726,40.927599,-73.997361,Nick's Pizza,4b8feb35f964a520886933e3,40.927700,-73.998180,Pizza Place
3,Bergenfield,726,40.927599,-73.997361,Cusinera,4c26424a136d20a1a1fee461,40.925610,-73.996678,Asian Restaurant
4,Bergenfield,726,40.927599,-73.997361,Bamboo Grill,4b7c8698f964a520ce982fe3,40.925985,-73.996648,Asian Restaurant
5,Bergenfield,726,40.927599,-73.997361,Papa John's Pizza,4c166694a9c220a15a945a9d,40.926377,-73.997253,Pizza Place
6,Bergenfield,726,40.927599,-73.997361,Asian-American Food Mart,4c0e78ff512f76b00fdd7a11,40.928696,-73.997985,Asian Restaurant
7,Bergenfield,726,40.927599,-73.997361,Nihon Kai Japanese Cuisine,4bd3725cb221c9b672ced9d0,40.926393,-73.996404,Japanese Restaurant
8,Bogota,162,40.876211,-74.029862,Mona Lisa's Cafe,4bff0767c30a2d7f7c68101d,40.878092,-74.029431,Pizza Place
9,Bogota,162,40.876211,-74.029862,La Famiglia,4bc9fa280687ef3b77ecdacc,40.876898,-74.024856,Italian Restaurant


In [93]:
# Restaurant clustering
bergen_open_restaurant = getRatingDistanceOpenStatus(boroughs=bergen_restaurant['Neighborhood'],
                                cases=bergen_restaurant['Coronavirus Positive Cases'],
                                venues=bergen_restaurant['Venue'],
                                venue_ids=bergen_restaurant['Venue id'],
                                venue_latitudes=bergen_restaurant['Venue Latitude'],
                                venue_longitudes=bergen_restaurant['Venue Longitude'],                        
                                categories=bergen_restaurant['Venue Category']
                                )
bergen_open_restaurant.head()

Chapala Grill 8.4 15093 Open until 10:30 PM
Empanada Mania 7.7 15093 Closed until Noon tomorrow
Nick's Pizza 7.5 15093 Closed until 11:00 AM tomorrow
Cusinera 7.4 15093 Closed until 8:00 AM Thursday
Bamboo Grill 6.7 15093 Closed until 11:00 AM tomorrow
Papa John's Pizza 6.5 15093 Closed until 11:00 AM tomorrow
Asian-American Food Mart 6.4 15093 N/A
Nihon Kai Japanese Cuisine 6.0 15093 N/A
Mona Lisa's Cafe 7.2 15087 N/A
La Famiglia 8.0 15087 Closed until Noon tomorrow
China Dragon N/A 15087 N/A
Pistahan N/A 15087 N/A
Hunkar Turkish Mediterranean Kebab House 8.3 15082 N/A
The Balcony Restaurant and Lounge 7.3 15082 Closed until Noon tomorrow
Bella Pizza 6.8 15082 N/A
Master Pizza N/A 15082 N/A
Rudy's Seafood Restaurant & Tavern 8.4 15083 Open until 11:30 PM
Doner Kebab House 8.4 15082 Closed until 11:00 AM tomorrow
Hakkı Baba 7.8 15082 Closed until Noon tomorrow
Vincent's Pizzeria 7.7 15082 Open until 11:00 PM
Silli Point Indian Fusion 7.6 15082 Closed until 11:30 AM tomorrow
Blackbeard 

The Risotto House Of Hasbrouck Heights N/A 15085 Closed until Noon tomorrow
Alessandro's 7.0 15097 N/A
Naturoll N/A 15097 N/A
Matsu 8.4 15100 Closed until 11:30 AM tomorrow
Cinco de Mayo 7.8 15100 Closed until 10:00 AM tomorrow
Big Daddy's Pizza & Grill 7.5 15100 Closed until 11:00 AM tomorrow
Domani Restaurant 7.4 15100 N/A
Della Cucina Ristorante & Pizzeria 7.3 15100 Closed until 11:00 AM tomorrow
Cornerstone 7.1 15100 Open until 3:00 AM
Wendy’s 6.5 15100 Open until 11:00 PM
Cortina's Ristorante & Pizzeria N/A 15100 N/A
Ho-Ho-Kus Inn & Tavern 8.7 15098 Closed until 11:30 AM tomorrow
St. Eve's 8.4 15098 Closed until Noon tomorrow
Cafe Amici 8.3 15098 Closed until 11:00 AM tomorrow
Ho-Ho-Kus Sushi Cafe 7.8 15098 N/A
Sicilian Sun 6.7 15098 Closed until 4:30 PM tomorrow
Nakahara 8.3 15086 Closed until Noon tomorrow
Fontana Di Trevi 7.9 15086 Closed until 4:00 PM tomorrow
Miga 7.9 15087 N/A
BonChon Chicken 7.8 15087 Closed until 10:00 AM tomorrow
Dante's  Place 6.4 15087 Closed until 11:0

Tribos Peri Peri N/A 15088 N/A
Dosa House/biryani house N/A 15088 N/A
Dynasty Buffet N/A 15088 N/A
Golden Pub N/A 15088 N/A
Saddle River Inn N/A 15102 N/A
Giorgio's N/A 15102 N/A
Bergen Brick Oven Bar & Grill N/A 15086 N/A
I Gemelli Ristorante N/A 15086 N/A
Martinos Grocery N/A 15086 N/A
I Gemelli N/A 15086 N/A
Amarone Ristorante N/A 15088 N/A
Teaneck Sushi Buffet N/A 15089 N/A
Amarone N/A 15088 N/A
Green Garden Restaurant N/A 15089 N/A
Stanley's Deli And Pizza N/A 15089 N/A
Patio Grill N/A 15088 N/A
Europe Cafe & Grill N/A 15094 N/A
Axia Taverna N/A 15094 N/A
Mandoo Cafe N/A 15094 N/A
Simply Vietnamese N/A 15094 N/A
Max's Italian Grill N/A 15094 N/A
Brasserie Tenafly N/A 15093 N/A
Tavlin N/A 15093 N/A
Mr. Wok & Sushi N/A 15094 N/A
Kinara Cuisine of India N/A 15094 N/A
Foova N/A 15094 N/A
Pizza Nova N/A 15093 N/A
Palmer's Crossing Restaurant & Bar N/A 15093 N/A
Chick-fil-A N/A 15085 N/A
Chipotle Mexican Grill N/A 15085 N/A
The Halal Guys N/A 15085 N/A
Blaze Pizza N/A 15085 N/A
Sarku Ja

,Venue,Venue Category,Borough,Coronavirus Positive Cases,Rating,Distance,Venue Latitude,Venue Longitude,Open Status
0,Chapala Grill,Mexican Restaurant,Bergenfield,726,8.4,15093,40.926046,-73.996719,Open until 10:30 PM
1,Empanada Mania,Empanada Restaurant,Bergenfield,726,7.7,15093,40.925710,-73.996562,Closed until Noon tomorrow
2,Nick's Pizza,Pizza Place,Bergenfield,726,7.5,15093,40.927700,-73.998180,Closed until 11:00 AM tomorrow
3,Cusinera,Asian Restaurant,Bergenfield,726,7.4,15093,40.925610,-73.996678,Closed until 8:00 AM Thursday
4,Bamboo Grill,Asian Restaurant,Bergenfield,726,6.7,15093,40.925985,-73.996648,Closed until 11:00 AM tomorrow


In [94]:
bergen_open_restaurant.sort_values(['Coronavirus Positive Cases', 'Rating', 'Distance'], ascending=[True, False, True])

,Venue,Venue Category,Borough,Coronavirus Positive Cases,Rating,Distance,Venue Latitude,Venue Longitude,Open Status
101,Ho-Ho-Kus Inn & Tavern,American Restaurant,Ho-Ho-Kus,42,8.7,15098,40.997717,-74.109788,Closed until 11:30 AM tomorrow
102,St. Eve's,American Restaurant,Ho-Ho-Kus,42,8.4,15098,40.996394,-74.110246,Closed until Noon tomorrow
103,Cafe Amici,American Restaurant,Ho-Ho-Kus,42,8.3,15098,40.997266,-74.109164,Closed until 11:00 AM tomorrow
104,Sicilian Sun,Italian Restaurant,Ho-Ho-Kus,42,6.7,15098,40.996267,-74.110514,Closed until 4:30 PM tomorrow
23,Yasou Mykonos,Greek Restaurant,Demarest,45,8.4,15097,40.957303,-73.963558,Closed until 11:00 AM tomorrow
123,Madeleine's Petit Paris,French Restaurant,Northvale,50,7.5,15102,41.009255,-73.953387,Closed until Noon tomorrow
121,Segovia,Spanish Restaurant,Moonachie,60,8.3,15083,40.839917,-74.048571,Closed until 11:30 AM tomorrow
120,Bazzarelli Restaurant,Italian Restaurant,Moonachie,60,8.0,15083,40.840647,-74.047752,Closed until 11:00 AM tomorrow
124,Callahan's,Fast Food Restaurant,Norwood,65,7.6,15101,40.999402,-73.966976,Closed until 11:00 AM tomorrow
125,Gerry's Place,American Restaurant,Norwood,65,6.2,15101,40.996538,-73.956619,Open until Midnight


In [95]:
print('Total restaurants: {}'.format(bergen_restaurant.shape[0]))
print('Total open restaurants: {}'.format(bergen_open_restaurant.shape[0]))

Total restaurants: 475
Total open restaurants: 159


In [96]:
# set number of clusters
kclusters = 5

bergen_restaurant_clustering = bergen_open_restaurant[['Coronavirus Positive Cases', 'Rating', 'Distance']]

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bergen_restaurant_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 2, 1, 4, 0, 0], dtype=int32)

In [97]:
# add clustering labels
bergen_open_restaurant.insert(2, 'Cluster Labels', kmeans.labels_)

bergen_open_restaurant.head()


,Venue,Venue Category,Cluster Labels,Borough,Coronavirus Positive Cases,Rating,Distance,Venue Latitude,Venue Longitude,Open Status
0,Chapala Grill,Mexican Restaurant,2,Bergenfield,726,8.4,15093,40.926046,-73.996719,Open until 10:30 PM
1,Empanada Mania,Empanada Restaurant,2,Bergenfield,726,7.7,15093,40.925710,-73.996562,Closed until Noon tomorrow
2,Nick's Pizza,Pizza Place,2,Bergenfield,726,7.5,15093,40.927700,-73.998180,Closed until 11:00 AM tomorrow
3,Cusinera,Asian Restaurant,2,Bergenfield,726,7.4,15093,40.925610,-73.996678,Closed until 8:00 AM Thursday
4,Bamboo Grill,Asian Restaurant,2,Bergenfield,726,6.7,15093,40.925985,-73.996648,Closed until 11:00 AM tomorrow


In [99]:
# create map
map_restaurant_clusters = folium.Map(location=[county_latitude, county_longitude], zoom_start=12)

# add my home
folium.CircleMarker(
        [home_latitude, home_longitude],
        radius=9,
        popup='Home',
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_restaurant_clusters)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, venue, cluster, cases, rating in zip(bergen_open_restaurant['Venue Latitude'], bergen_open_restaurant['Venue Longitude'], bergen_open_restaurant['Venue'], bergen_open_restaurant['Cluster Labels'], bergen_open_restaurant['Coronavirus Positive Cases'], bergen_open_restaurant['Rating']):
    label = folium.Popup(str(venue) + ' [cluster: ' + str(cluster) + ', Coronavirus cases: ' + str(cases) + ', Rating: ' + str(rating) + ']', parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_restaurant_clusters)
       
map_restaurant_clusters

In [100]:
# restaurant - cluster 0 - medium coronuavirus cases
bergen_open_restaurant.loc[bergen_open_restaurant['Cluster Labels'] == 0, bergen_open_restaurant.columns[[0]+[1]+[3]+[4]+[5]+[6]]]

,Venue,Venue Category,Borough,Coronavirus Positive Cases,Rating,Distance
8,Rudy's Seafood Restaurant & Tavern,Italian Restaurant,Cliffside Park,415,8.4,15083
9,Doner Kebab House,Doner Restaurant,Cliffside Park,415,8.4,15082
10,Hakkı Baba,Turkish Restaurant,Cliffside Park,415,7.8,15082
11,Vincent's Pizzeria,Pizza Place,Cliffside Park,415,7.7,15082
12,Silli Point Indian Fusion,Indian Restaurant,Cliffside Park,415,7.6,15082
13,Blackbeard Ramen,Ramen Restaurant,Cliffside Park,415,7.8,15083
14,Petite Soochow | 小苏州,Chinese Restaurant,Cliffside Park,415,7.9,15083
15,House Malaysian,Malay Restaurant,Cliffside Park,415,7.9,15083
16,Szechaun Mirchi,Asian Restaurant,Cliffside Park,415,7.0,15083
63,Wok Bar,Asian Restaurant,Fort Lee,386,8.0,15086


In [101]:
# restaurant - cluster 1 - low coronuavirus cases
bergen_open_restaurant.loc[bergen_open_restaurant['Cluster Labels'] == 1, bergen_open_restaurant.columns[[0]+[1]+[3]+[4]+[5]+[6]]]

,Venue,Venue Category,Borough,Coronavirus Positive Cases,Rating,Distance
6,La Famiglia,Italian Restaurant,Bogota,162,8.0,15087
29,Caffe Capri,Italian Restaurant,East Rutherford,123,8.5,15081
30,Papa John's Pizza,Pizza Place,East Rutherford,123,6.2,15081
31,Duck King Chinese Cuisine,Chinese Restaurant,Edgewater,119,7.9,15083
32,The Great Burrito,Mexican Restaurant,Edgewater,119,7.9,15083
33,River Thai Food,Thai Restaurant,Edgewater,119,7.5,15083
34,Jack's Lobster Shack,Seafood Restaurant,Edgewater,119,8.7,15083
35,Marmaris Cafe,Turkish Restaurant,Edgewater,119,8.4,15083
36,Roberto's II Ristorante,Italian Restaurant,Edgewater,119,7.2,15083
37,Arigato,Sushi Restaurant,Edgewater,119,7.5,15083


In [102]:
# restaurant - cluster 2 - high coronuavirus cases
bergen_open_restaurant.loc[bergen_open_restaurant['Cluster Labels'] == 2, bergen_open_restaurant.columns[[0]+[1]+[3]+[4]+[5]+[6]]]

,Venue,Venue Category,Borough,Coronavirus Positive Cases,Rating,Distance
0,Chapala Grill,Mexican Restaurant,Bergenfield,726,8.4,15093
1,Empanada Mania,Empanada Restaurant,Bergenfield,726,7.7,15093
2,Nick's Pizza,Pizza Place,Bergenfield,726,7.5,15093
3,Cusinera,Asian Restaurant,Bergenfield,726,7.4,15093
4,Bamboo Grill,Asian Restaurant,Bergenfield,726,6.7,15093
5,Papa John's Pizza,Pizza Place,Bergenfield,726,6.5,15093
45,Cassie's Restaurant,Pizza Place,Englewood,690,8.6,15090
46,Akai Lounge,Japanese Restaurant,Englewood,690,8.5,15090
47,Bennies,Middle Eastern Restaurant,Englewood,690,8.5,15090
48,Blue Moon Mexican Cafe,Mexican Restaurant,Englewood,690,7.8,15090


In [103]:
# restaurant - cluster 3 - medium coronuavirus cases
bergen_open_restaurant.loc[bergen_open_restaurant['Cluster Labels'] == 3, bergen_open_restaurant.columns[[0]+[1]+[3]+[4]+[5]+[6]]]

,Venue,Venue Category,Borough,Coronavirus Positive Cases,Rating,Distance
24,Uncle Frank's Pizza,Pizza Place,Dumont,278,8.1,15094
25,Italian Connection Pizza,Pizza Place,Dumont,278,7.7,15094
26,Cuban Eddies,Cuban Restaurant,Dumont,278,8.0,15095
27,Il Mulino Restaurant,Italian Restaurant,Dumont,278,7.3,15094
28,Domino's Pizza,Pizza Place,Dumont,278,6.8,15094
61,White Castle,Fast Food Restaurant,Fairview,315,8.2,15081
62,La Grande Buffet,Restaurant,Fairview,315,7.5,15081
115,Seafood Gourmet,Seafood Restaurant,Maywood,201,8.7,15089
116,Uncle Paulie's Puro Sabor,Latin American Restaurant,Maywood,201,8.5,15089
117,Mario's Pizza,Pizza Place,Maywood,201,8.4,15089


In [104]:
# restaurant - cluster 4 - low coronuavirus cases
bergen_open_restaurant.loc[bergen_open_restaurant['Cluster Labels'] == 4, bergen_open_restaurant.columns[[0]+[1]+[3]+[4]+[5]+[6]]]

,Venue,Venue Category,Borough,Coronavirus Positive Cases,Rating,Distance
7,The Balcony Restaurant and Lounge,Restaurant,Carlstadt,89,7.3,15082
17,Zendiggi Kebab House,Middle Eastern Restaurant,Closter,72,8.1,15098
18,Ma Mi,Vietnamese Restaurant,Closter,72,7.8,15098
19,Carmine & Ray's Pizza,Pizza Place,Closter,72,6.3,15098
20,Farmhouse Cafe & Eatery,American Restaurant,Cresskill,96,8.3,15095
21,Hanami,Chinese Restaurant,Cresskill,96,7.8,15095
22,Umeya,Sushi Restaurant,Cresskill,96,7.7,15095
23,Yasou Mykonos,Greek Restaurant,Demarest,45,8.4,15097
83,John's Boy Pizzeria,Pizza Place,Glen Rock,99,8.7,15094
84,Francesca Brick Oven Pizza,Pizza Place,Glen Rock,99,8.2,15094
